In [ ]:
import re
import datefinder
import string
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from textblob import TextBlob

In [ ]:
## Journal.txt should be in this format:
#########
#Day 1
#Date/time
#Action: text here
#Inspiration: text here
#Why not:
#	1. Why not do it
#		a. Why yes do it
#	2. Why not do it
#		a. Why yes do it
#Feelings:
#text here
#Day 2
#Date/time
#Action: text here
#Inspiration: text here
#Why not:
#	1. Why not do it
#		a. Why yes do it
#	2. Why not do it
#		a. Why yes do it
#Feelings:
#text here
############

In [ ]:
#Import journal into x, you will need to change the name of the file "Journal1.txt"
x = "Journal1.txt"
#Open text file in read ('r') mode and remove \n (new line) indications
y = open(x, 'r').read().replace('\n', '')

In [ ]:
#Split the journal by days
Days=re.split('Day ',y)

In [ ]:
#Allocate lists for parameters to fill
Action=[None]*(len(Days)-1)
Inspiration=[None]*(len(Days)-1)
#list of why nots
WN=[None]*(len(Days)-1)
Feelings=[None]*(len(Days)-1)
#number of why nots per day
numWN=[None]*(len(Days)-1)

#split text into their parameters
for i in range(1,len(Days)):
    Action[i-1]=re.split('Action: ',re.split('Inspiration',Days[i])[0])[1]
    Inspiration[i-1]=re.split('Inspiration:',re.split('Why nots',Days[i])[0])[1]
    WN[i-1]=re.split('Why not:',re.split('Feelings',Days[i])[0])[1]
    Feelings[i-1]=re.split('Feelings:',Days[i])[1]
    numWN[i-1]=WN[i-1].count('\t')-2*WN[i-1].count('\t\t')

In [ ]:
#Additional organization of WN and WY
WN_list=[None]*(len(Days)-1)
WNWY_list=[None]*(len(Days)-1)
WY_list=[None]*(len(Days)-1)
lets=[None]*(len(Days)-1)
nxt=[None]*(len(Days)-1)

In [ ]:
for j in range(0,len(Days)-1):
    WN_list[j]=[None]*numWN[j]
    WNWY_list[j]=[None]*numWN[j]
    WY_list[j]=[None]*numWN[j]
    lets[j]=[None]*numWN[j]
    for i in range(1,numWN[j]+1):
        nxt[j]=i+1;
        WNWY_list[j][i-1]=re.split('\t%d. ' %nxt[j],re.split('\t%d. ' %i,WN[j])[1])[0]
        WN_list[j][i-1]=re.split('\t',re.split('\t%d. ' %nxt[j],re.split('\t%d. ' %i,WN[j])[1])[0])[0]
        
    for i in range(1,numWN[j]+1):        
        WY_list[j][i-1]=re.split('\t\ta. ', WNWY_list[j][i-1])[1]

In [ ]:
list2=[Action, Inspiration, WN_list, WY_list, Feelings]

In [ ]:
#Put entries into data frame
df=pd.DataFrame(data=list2).transpose().rename(columns={0:'Action', 1:'Inspiration', 2:'No', 3:'Yes', 4:'Feelings'})

In [ ]:
#Combine no's and yes into columns
for i in range(0,len(df)):
    df['No'][i]=' '.join(df['No'][i])
    df['Yes'][i]=' '.join(df['Yes'][i])

In [ ]:
#This section is specific to my journal entries, I was making sure the words one last talk 
#always went together as this is a title of a book I was reading
Nos=' '.join(df['No']).lower().replace('one last talk', 'one_last_talk')
Yes=' '.join(df['Yes']).lower().replace('one last talk', 'one_last_talk')
Inspiration_all=df['Inspiration'].str.cat(sep=' ').lower().replace('one last talk', 'one_last_talk')
Action_all=df['Action'].str.cat(sep=' ').lower().replace('one last talk', 'one_last_talk')
Feelings_all=df['Feelings'].str.cat(sep=' ').lower().replace('one last talk', 'one_last_talk')

In [ ]:
#You can enter stopwords here (words you want to exclude from the wordcloud) I've left a few as examples
STOPWORDS.add('will')
STOPWORDS.add('got')

In [ ]:
#Action wordcloud
wordcloud=WordCloud(background_color="white",stopwords=STOPWORDS,max_words=50,width=600,height=400,colormap='winter').generate(Action_all)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
#Inspiration word cloud
wordcloud=WordCloud(background_color="white",stopwords=STOPWORDS,max_words=50,width=600,height=400).generate(Inspiration_all)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
#Why not word cloud
wordcloud=WordCloud(background_color="white",max_words=40,width=600,height=400,colormap='winter').generate(Nos)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
#Why yes word cloud
wordcloud=WordCloud(background_color="white",max_words=50,width=600,height=400).generate(Yes)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
#Feelings word cloud
wordcloud=WordCloud(background_color="white",max_words=50,width=600,height=400).generate(Feelings_all)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
#Polarity analysis
#There wasn't really enough data to do this, but I've left it in the code in case somebody finds it useful
polarity_feelings=[None]*len(Feelings)
polarity_action=[None]*len(Feelings)
polarity_inspiration=[None]*len(Feelings)
polarity_yes=[None]*len(Feelings)
polarity_nos=[None]*len(Feelings)
for i in range(0,len(Feelings)):
    polarity_feelings[i]=TextBlob(Feelings[i]).sentiment.polarity
    polarity_action[i]=TextBlob(Action[i]).sentiment.polarity
    polarity_inspiration[i]=TextBlob(Inspiration[i]).sentiment.polarity
    polarity_yes[i]=TextBlob(df['Yes'][i]).sentiment.polarity
    polarity_nos[i]=TextBlob(df['No'][i]).sentiment.polarity

In [ ]:
plt.plot(polarity_feelings)

In [ ]:
plt.plot(polarity_action)

In [ ]:
plt.plot(polarity_inspiration)

In [ ]:
plt.plot(polarity_yes)

In [ ]:
plt.plot(polarity_nos)

In [ ]:
plt.plot(numWN)

In [ ]:
#You can also look at the length of each section of each entry. Since feelings was a free-form section this made the most sense.
#Again, not really enough data to draw any conclusions but I've left it here in case.
lenfeel=[None]*len(Feelings)
for i in range(0,len(Feelings)):
    lenfeel[i]=len(Feelings[i])

In [ ]:
plt.plot(lenfeel)